In [1]:
#Import dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
from pprint import pprint
from urllib.request import urlopen, Request
from datetime import datetime
from config import password

In [2]:
#Decided to find a website to scrape the state abbreviations
url_abv = "https://abbreviations.yourdictionary.com/articles/state-abbrev.html"

In [3]:
#call data and use Beautiful Soup to parse
response = requests.get(url_abv)
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
#HTML inspect found the values in the li tag
results = soup.find_all('li')

In [5]:
#strip out only the first 50 li tags that contain the abbreviations
ABV = []

In [6]:
for i in range(0, 50):
    results[i].text
    ABV.append(results[i].text)

In [7]:
#Take the new lists and seperate the abbreviations from state names into a new lists
ABV_new = []
State_name = []

In [8]:
#abbreviation list
for i in range(0, 50):
    ABV_clean = ABV[i].split(' - ')[1]
    ABV_new.append(ABV_clean.replace(' ',''))

In [9]:
# #state name list
# for i in range(0, 50):
#     ABV_clean = ABV[i].split(' - ')[0]
#     State_name.append(ABV_clean.replace(' ',''))

In [10]:
#create dataframe
All_States_df = pd.DataFrame({'Abbreviation':ABV_new})
All_States_df.head()

,Abbreviation
0,AL
1,AK
2,AZ
3,AR
4,CA


In [11]:
#for MAC
# print out the location of the driver
# !which chromedriver

# identify location of chromedriver and store it as a variable
# driverPath = !which chromedriver

# # Setup configuration variables to enable Splinter to interact with browser
# executable_path = {'executable_path': driverPath[0]}
# browser = Browser('chrome', **executable_path, headless=False)

In [12]:
#Get chromedriver and splinter  up and running for PC
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [13]:
#set url for the article, paragraph, and image

url="https://www.bls.gov/charts/state-employment-and-unemployment/state-unemployment-rates-animated.htm"
browser.visit(url)

In [14]:
#click table
browser.click_link_by_partial_text('Show table')

In [15]:
#read html table as pandas
RawData = browser.html
tables = pd.read_html(RawData)

In [16]:
df = tables[0]
clean_df = df.drop([8,51])
#set index to state to make sure all columns are dates
clean_df = clean_df.set_index("State")

#convert all column lables to datetime and format as month_year
clean_df.columns = pd.to_datetime(clean_df.columns).strftime("%b-%Y")

#reset index to get state as it's own column
clean_df = clean_df.reset_index("State")
result = pd.concat([clean_df, All_States_df], axis=1, sort=False)

del result['State']
result.rename(columns = {"Abbreviation": "state"}, 
                                 inplace = True) 

result = result.set_index("state")

result.head()

,Feb-2009,Mar-2009,Apr-2009,May-2009,Jun-2009,Jul-2009,Aug-2009,Sep-2009,Oct-2009,Nov-2009,...,May-2018,Jun-2018,Jul-2018,Aug-2018,Sep-2018,Oct-2018,Nov-2018,Dec-2018,Jan-2019,Feb-2019
state,,,,,,,,,,,,,,,,,,,,,
AL,9.6,10.2,10.7,11.0,11.3,11.5,11.6,11.7,11.8,11.8,...,4.1,4.1,4.0,3.9,3.8,3.8,3.8,3.8,3.8,3.7
AK,7.3,7.4,7.6,7.7,7.8,7.9,8.0,8.0,8.0,8.0,...,6.6,6.6,6.6,6.5,6.5,6.5,6.5,6.5,6.5,6.5
AZ,8.8,9.1,9.4,9.7,9.9,10.2,10.4,10.6,10.8,10.9,...,4.7,4.7,4.7,4.8,4.8,4.9,4.9,4.9,5.1,5.1
AR,7.4,7.6,7.8,7.9,7.9,7.9,8.0,8.0,8.0,8.1,...,3.7,3.7,3.6,3.6,3.6,3.7,3.7,3.7,3.7,3.8
CA,10.1,10.5,10.8,11.0,11.2,11.4,11.5,11.7,11.8,11.9,...,4.2,4.2,4.1,4.1,4.1,4.1,4.1,4.1,4.2,4.2


In [17]:
unemploy_df = result.stack().reset_index()
unemploy_df = unemploy_df.rename(columns={'level_1': 'month_year', 
                                          0: 'unemployment_data'})

# unemploy_df["month_year"] = pd.to_datetime(unemploy_df["month_year"])

unemploy_df = unemploy_df.set_index(['state', 'month_year'], inplace=False)

unemploy_df.head()

unemployment_data
state month_year                   
AL    Feb-2009                  9.6
      Mar-2009                 10.2
      Apr-2009                 10.7
      May-2009                 11.0
      Jun-2009                 11.3

In [18]:
#import mysql dependencies
import pymysql
from config import password
pymysql.install_as_MySQLdb()

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String, Float

from sqlalchemy import create_engine

In [19]:
#write unemployment dataframe to mySQL
dbuser = 'root'
dbpassword = password
dbhost = 'localhost'
dbport = '3306'
dbname= 'project2'

engine = create_engine(f"mysql://{dbuser}:{dbpassword}@{dbhost}:{dbport}/{dbname}")
unemploy_df.to_sql('unemployment', engine, index=True, if_exists="append")

In [20]:
#import dependencies for drug data
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
from datetime import datetime

#import Socrata to pull API data...may need to run pip install sodapy
from sodapy import Socrata

#import password for mySQL from config.py.
from config import password

# import app_token (if desired) from app_token.py to prevent limited pull requests
# from app_token import app_token

In [22]:
#call Socrata without app_token
client = Socrata("data.cdc.gov", None)

# # Example authenticated client (needed for non-public datasets):
# client = Socrata("data.cdc.gov", app_token)

#requests result, limit is 25000 to ensure full data is pulled
results = client.get("a3uk-kgrx", limit="25000")

# Convert to pandas DataFrame
drug_df = pd.DataFrame.from_records(results)

#create month and year string from month and year columns
drug_df["month_year"] = drug_df["month"].apply(lambda x: x[:3]) + "-" + drug_df["year"]

#converty month_year column to datetime

# drug_df["month_year"] = pd.to_datetime(drug_df["month_year"])


#drop duplicate entries from dataframe
drug_df = drug_df.drop_duplicates(subset=['indicator', 'state', 'month_year'])
drug_df = drug_df.reset_index(drop=True)


In [23]:
#group dataframe by state and month_year
drug_df=drug_df.sort_values(by=['state','month_year'])
drug_df = drug_df.set_index(['state', 'month_year'], inplace=False)

#pivot values based on indicators, grouped by state and month_year
drug_pivot = drug_df.pivot_table(values='data_value', index=['state', 'month_year'], columns='indicator', aggfunc=np.sum)

#rename columns
drug_pivot = drug_pivot.rename(columns={"Cocaine (T40.5)": 'cocaine', "Heroin (T40.1)" : "heroin", "Methadone (T40.3)": "methadone", "Opioids (T40.0-T40.4,T40.6)": "opioids",
                                       "Psychostimulants with abuse potential (T43.6)": "psychostimulants"})

#drop unneeded columns
drug_pivot = drug_pivot.drop(['Natural & semi-synthetic opioids (T40.2)', 'Natural & semi-synthetic opioids, incl. methadone (T40.2, T40.3)',"Natural, semi-synthetic, & synthetic opioids, incl. methadone (T40.2-T40.4)",
                             "Synthetic opioids, excl. methadone (T40.4)"], axis=1)

#rename remaning columns with underscores so js friendly
drug_pivot = drug_pivot.rename(columns={"Number of Deaths": 'number_of_deaths', "Number of Drug Overdose Deaths" : "number_drug_overdose_death", 
                                        "Percent with drugs specified": "percent_drugs_specified"})


In [24]:
#push dataframe contents to mysql

import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine

dbuser = 'root'
#read in password from config.py file
dbpassword = password
dbhost = 'localhost'
dbport = '3306'

#database name is Project2 (need to create this locally)
dbname= 'project2'

engine = create_engine(f"mysql://{dbuser}:{dbpassword}@{dbhost}:{dbport}/{dbname}")

# stat_cons_df
# drug_df.to_sql('drug', engine, index=True, if_exists="append")
drug_pivot.to_sql('drug', engine, index=True, if_exists="append")

In [25]:
#create combined dataframe and push to sql
combined_df = drug_pivot.join(unemploy_df)
combined_df.to_sql('combined', engine, index=True, if_exists="append")

In [26]:
combined_df

cocaine heroin methadone number_of_deaths  \
state month_year                                             
AK    Apr-2015       NaN    NaN       NaN             4133   
      Apr-2016       NaN    NaN       NaN             4262   
      Apr-2017       NaN    NaN       NaN             4325   
      Apr-2018       NaN    NaN       NaN             4341   
      Aug-2015       NaN    NaN       NaN             4222   
      Aug-2016       NaN    NaN       NaN             4276   
      Aug-2017       NaN    NaN       NaN             4234   
      Aug-2018       NaN    NaN       NaN             4382   
      Dec-2015       NaN    NaN       NaN             4193   
      Dec-2016       NaN    NaN       NaN             4355   
      Dec-2017       NaN    NaN       NaN             4279   
      Feb-2015       NaN    NaN       NaN             4084   
      Feb-2016       NaN    NaN       NaN             4198   
      Feb-2017       NaN    NaN       NaN             4347   
      Feb-2018       NaN    NaN       NaN             4304   
      Jan-2015       NaN    NaN       NaN             4034   
      Jan-2016       NaN    NaN       NaN             4213   
      Jan-2017       NaN    NaN       NaN             4374   
      Jan-2018       NaN    NaN       NaN             4260   
      Jul-2015       NaN    NaN       NaN             4220   
      Jul-2016       NaN    NaN       NaN             4269   
      Jul-2017       NaN    NaN       NaN             4236   
      Jul-2018       NaN    NaN       NaN             4337   
      Jun-2015       NaN    NaN       NaN             4201   
      Jun-2016       NaN    NaN       NaN             4261   
      Jun-2017       NaN    NaN       NaN             4276   
      Jun-2018       NaN    NaN       NaN             4309   
      Mar-2015       NaN    NaN       NaN             4101   
      Mar-2016       NaN    NaN       NaN             4230   
      Mar-2017       NaN    NaN       NaN             4359   
...                  ...    ...       ...              ...   
YC    Feb-2018       644    650       190            54919   
      Jan-2015       231    299       112            53516   
      Jan-2016       308    421       125            53624   
      Jan-2017       537    597       178            54566   
      Jan-2018       638    644       177            54737   
      Jul-2015       242    322       129            54291   
      Jul-2016       402    517       149            53502   
      Jul-2017       599    616       166            54200   
      Jul-2018       662    640       180            55034   
      Jun-2015       244    307       124            54107   
      Jun-2016       370    501       143            53435   
      Jun-2017       601    620       170            54245   
      Jun-2018       650    659       180            54998   
      Mar-2015       221    292       106            54149   
      Mar-2016       344    458       141            53547   
      Mar-2017       541    584       166            54145   
      Mar-2018       648    671       193            54993   
      May-2015       230    306       119            54029   
      May-2016       361    482       143            53496   
      May-2017       585    602       165            54194   
      May-2018       644    675       188            54916   
      Nov-2015       283    402       132            54320   
      Nov-2016       475    547       165            53882   
      Nov-2017       631    647       168            54241   
      Oct-2015       272    375       131            54572   
      Oct-2016       465    546       161            53545   
      Oct-2017       620    644       163            54287   
      Sep-2015       266    360       136            54449   
      Sep-2016       442    540       147            53533   
      Sep-2017       615    634       172            54331   

                 number_drug_overdose_death opioids percent_drugs_specified  \
state month_year               

In [27]:
#test space to convert to dictionary for Flask_API.py
test_df = combined_df.reset_index()
test_df.head()

,state,month_year,cocaine,heroin,methadone,number_of_deaths,number_drug_overdose_death,opioids,percent_drugs_specified,psychostimulants,unemployment_data
0,AK,Apr-2015,NaN,NaN,NaN,4133,126,NaN,NaN,NaN,6.5
1,AK,Apr-2016,NaN,NaN,NaN,4262,132,NaN,NaN,NaN,6.8
2,AK,Apr-2017,NaN,NaN,NaN,4325,117,NaN,NaN,NaN,7.0
3,AK,Apr-2018,NaN,NaN,NaN,4341,141,NaN,NaN,NaN,6.7
4,AK,Aug-2015,NaN,NaN,NaN,4222,124,NaN,NaN,NaN,6.5


In [28]:
#groupby state and month_year
datagrouped = test_df.groupby(['state', 'month_year'])

#create empty dataframe
group_df = pd.DataFrame()

In [29]:
#list all columns needed for database
columns = ['cocaine', 'heroin', 'methadone', 'number_of_deaths', 'number_drug_overdose_death', 'opioids', 'percent_drugs_specified', 'psychostimulants', "unemployment_data"]

In [30]:
#extract groupby objects using .sum() and create new dataframe columns
for column in columns:
    group_df[f"{column}"] = datagrouped[f"{column}"].sum()

In [32]:
group_df.to_csv('combined.csv')

In [ ]:
# #convert dataframe to dictionary list
# group_df.groupby(level=0).apply(lambda df: df.xs(df.name).to_dict()).to_dict()